In [41]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import models
import numpy as np
import pandas as pd 
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

### Load Best Model

In [42]:
# Create a feedforward NN with:
# 1 hidden layer with self.first_hidden_size neurons and Relu activation function.
# 1 output layer with 3 neurons and softmax activation function.
class MLP(torch.nn.Module):
        def __init__(self, input_size, first_hidden_size):
            super(MLP, self).__init__()
            self.input_size = input_size
            self.first_hidden_size  = first_hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.first_hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.first_hidden_size, 3)
            self.softmax = torch.nn.Softmax(dim = 1)
            
            
        def forward(self, x):
            first_hidden = self.fc1(x)
            first_relu = self.relu(first_hidden)
            
            output = self.fc2(first_relu)
            output = self.softmax(output)
            return output   

### Test MLP

In [43]:
# Computes the accuracy.
def compute_accuracy(predictions, labels):
    # Take the max value and the max index of every vector in predictions.
    max_values, max_indexes = torch.max(predictions.data, 1)
    # Compute the accuracy.
    accuracy = (((max_indexes == labels).sum().item()) / labels.shape[0]) * 100
    return accuracy

def test_model(model, data, actual_labels):
    # Set evaluation mode.
    model.eval()
    
    # Compute predicted labels.
    predicted_test_labels = model(data)
    
    # Compute accuracy.
    final_accuracy = compute_accuracy(predicted_test_labels, actual_labels)
    
    return final_accuracy, predicted_test_labels

In [44]:
# A list that has the names of all cases.
cases_folder_names = ['\\case01', '\\case02', '\\case03', '\\case04', '\\case05', '\\case06', '\\case07', '\\case08', 
                      '\\case09', '\\case10', '\\case11', '\\case12', '\\case13', '\\case14', '\\case15', '\\case16',
                      '\\case17', '\\case18', '\\case19', '\\case20', '\\case21', '\\case22', '\\case23']

embeddings_directory = r"C:\Nikolaos Sintoris\Education\MEng CSE - UOI\Diploma Thesis\Training Results\Layer 4\Embeddings\ResNet50 Classifier"
best_model_directory = r"C:\Nikolaos Sintoris\Education\MEng CSE - UOI\Diploma Thesis\Training Results\Layer 4\Best Model\ResNet50 Classifier"
for current_case_name in cases_folder_names:
    print("Current Case Name: ", current_case_name)
    
    #===============================================================
    #==================== Load Test Dataset ==========================
    current_case_embeddings_directory = embeddings_directory + current_case_name
    
    test_data_np = np.loadtxt(current_case_embeddings_directory + '\\test_data.csv', delimiter = ',')
    test_data = torch.from_numpy(test_data_np) # It does not create a copy. Uses the same memory.
    test_data = test_data.float()
    print("\tTest data shape: ", test_data.shape)

    actual_test_labels_np = np.loadtxt(current_case_embeddings_directory + '\\actual_test_labels.csv', delimiter = ',')
    actual_test_labels = torch.from_numpy(actual_test_labels_np) # It does not create a copy. Uses the same memory.
    actual_test_labels = actual_test_labels.long()
    print("\tActual test labels shape: ", actual_test_labels.shape)
    #===============================================================
    #===============================================================
    
    #===============================================================
    #==================== Load Best Model ==========================
    current_case_best_model_directory = best_model_directory + current_case_name + "\\state_dict_model.pt"
    
    # Load the model with the maximum validation accuracy.
    input_layer_size = test_data.shape[1]
    first_hidden_layer_size = round(math.sqrt(input_layer_size * 3))
    my_model = MLP(input_layer_size, first_hidden_layer_size)
    my_model.load_state_dict(torch.load(current_case_best_model_directory))
    #===============================================================
    #===============================================================
    
    #===============================================================
    #==================== Compute Accuracy ==========================
    test_accuracy, predicted_test_labels = test_model(my_model, test_data, actual_test_labels)
    print("\tTest accuracy: ", test_accuracy)
    #===============================================================
    #===============================================================
    
    #===============================================================
    #==================== Compute F1 Score ==========================
    _, final_predicted_test_labels = torch.max(predicted_test_labels.data, 1)
    final_predicted_test_labels_np = final_predicted_test_labels.numpy()

    test_f1_score = f1_score(actual_test_labels_np, final_predicted_test_labels_np, average = 'macro')
    print("\tTest F1-Score: ", test_f1_score)
    #===============================================================
    #===============================================================
    
    #===============================================================
    #==================== Save Results ==========================
    # Save test accuracy and F1-score to a csv file.
    test_results_np = np.array( [test_accuracy, test_f1_score] )
    np.savetxt(best_model_directory + current_case_name + "\\accuracy_f1_score.csv", test_results_np, delimiter = ',')
    #===============================================================
    #===============================================================

Current Case Name:  \case01
	Test data shape:  torch.Size([813, 2048])
	Actual test labels shape:  torch.Size([813])
	Test accuracy:  91.26691266912668
	Test F1-Score:  0.7861918891219609
Current Case Name:  \case02
	Test data shape:  torch.Size([609, 2048])
	Actual test labels shape:  torch.Size([609])
	Test accuracy:  61.412151067323485
	Test F1-Score:  0.5891802615189787
Current Case Name:  \case03
	Test data shape:  torch.Size([1305, 2048])
	Actual test labels shape:  torch.Size([1305])
	Test accuracy:  89.65517241379311
	Test F1-Score:  0.6501738726074174
Current Case Name:  \case04
	Test data shape:  torch.Size([609, 2048])
	Actual test labels shape:  torch.Size([609])
	Test accuracy:  98.0295566502463
	Test F1-Score:  0.8832184197039
Current Case Name:  \case05
	Test data shape:  torch.Size([753, 2048])
	Actual test labels shape:  torch.Size([753])
	Test accuracy:  89.9070385126162
	Test F1-Score:  0.8601644607546021
Current Case Name:  \case06
	Test data shape:  torch.Size([498